In [1]:
import json
import os
import time
from pprint import pprint

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from IPython.display import clear_output
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torch import optim
from torchvision import transforms

%matplotlib inline

# Train model

## Install yolo5 library

In [2]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 14156, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 14156 (delta 56), reused 74 (delta 43), pack-reused 14060
Receiving objects: 100% (14156/14156), 13.54 MiB | 537.00 KiB/s, done.
Resolving deltas: 100% (9719/9719), done.


In [3]:
!pip install -qr ./yolov5/requirements.txt

In [4]:
from yolov5 import train, val, detect
from yolov5.utils import metrics

## Prepare data config file

In [5]:
DATA_YAML_PATH = 'data.yaml'

In [6]:
_ = os.system(f'cat {DATA_YAML_PATH}')

path: /home/maxim/Desktop/2022-2023/icv/

train: data/train/images
val: data/validation/images
test: data/test/images

names:
  0: animal


## Prepare parameters config file

In [7]:
HYP_YAML_PATH = 'hyps/default.yaml'

In [8]:
_ = os.system(f'cat {HYP_YAML_PATH}')

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.01  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)


## Run training

In [9]:
NUM_EPOCHS = 1
BATCH_SIZE = 15
OPTIMIZER = 'Adam'

In [10]:
train.run(
    data=DATA_YAML_PATH,
    hyp=HYP_YAML_PATH,
    
    epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    optimizer=OPTIMIZER,
    
    single_cls=True,
    noplots=False,
    tensorboard=True,
    device='cuda:0'
)

train: weights=yolov5/yolov5s.pt, cfg=, data=data.yaml, hyp=hyps/default.yaml, epochs=1, batch_size=15, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=cuda:0, multi_scale=False, single_cls=True, optimizer=Adam, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, tensorboard=True
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-8-g350e8eb Python-3.10.6 torch-1.13.0+cu117 CUDA:0 (NVIDIA GeForce GTX 1050, 4040MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, h

  0%|          | 0.00/14.1M [00:00<?, ?B/s]


Overriding model.yaml nc=80 with nc=1

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]            

Namespace(weights='yolov5/yolov5s.pt', cfg='', data='data.yaml', hyp={'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'warmup_bias_lr': 0.1, 'box': 0.05, 'cls': 0.5, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0}, epochs=1, batch_size=15, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket='', cache=None, image_weights=False, device='cuda:0', multi_scale=False, single_cls=True, optimizer='Adam', sync_bn=False, workers=8, project='yolov5/runs/train', name='exp', exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_datas